In [304]:
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [305]:
df= pd.read_csv('movies.csv')
df.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [306]:
df.shape


(4803, 24)

In [307]:
selected_feature=['genres','keywords','tagline','cast','director']
selected_feature

['genres', 'keywords', 'tagline', 'cast', 'director']

In [308]:
df.isnull().sum()

index                      0
budget                     0
genres                    28
homepage                3091
id                         0
keywords                 412
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
cast                      43
crew                       0
director                  30
dtype: int64

In [309]:
for feature in selected_feature :
    df[feature]=df[feature].fillna('')

In [310]:
df.isnull().sum()

index                      0
budget                     0
genres                     0
homepage                3091
id                         0
keywords                   0
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                    0
title                      0
vote_average               0
vote_count                 0
cast                       0
crew                       0
director                   0
dtype: int64

In [311]:
combined_feature = df[['genres','keywords','tagline','cast']]
combined_feature

,genres,keywords,tagline,cast
0,Action Adventure Fantasy Science Fiction,culture clash future space war space colony so...,Enter the World of Pandora.,Sam Worthington Zoe Saldana Sigourney Weaver S...
1,Adventure Fantasy Action,ocean drug abuse exotic island east india trad...,"At the end of the world, the adventure begins.",Johnny Depp Orlando Bloom Keira Knightley Stel...
2,Action Adventure Crime,spy based on novel secret agent sequel mi6,A Plan No One Escapes,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...
3,Action Crime Drama Thriller,dc comics crime fighter terrorist secret ident...,The Legend Ends,Christian Bale Michael Caine Gary Oldman Anne ...
4,Action Adventure Science Fiction,based on novel mars medallion space travel pri...,"Lost in our world, found in another.",Taylor Kitsch Lynn Collins Samantha Morton Wil...
...,...,...,...,...
4798,Action Crime Thriller,united states\u2013mexico barrier legs arms pa...,"He didn't come looking for trouble, but troubl...",Carlos Gallardo Jaime de Hoyos Peter Marquardt...
4799,Comedy Romance,,A newlywed couple's honeymoon is upended by th...,Edward Burns Kerry Bish\u00e9 Marsha Dietlein ...
4800,Comedy Drama Romance TV Movie,date love at first sight narration investigati...,,Eric Mabius Kristin Booth Crystal Lowe Geoff G...
4801,,,A New Yorker in Shanghai,Daniel Henney Eliza Coupe Bill Paxton Alan Ruc...


In [312]:
combined =df['genres']+' ' +df['keywords']+""+df['tagline']+""+df['cast']+""+df['director']
combined

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4798    Action Crime Thriller united states\u2013mexic...
4799    Comedy Romance A newlywed couple's honeymoon i...
4800    Comedy Drama Romance TV Movie date love at fir...
4801     A New Yorker in ShanghaiDaniel Henney Eliza C...
4802    Documentary obsession camcorder crush dream gi...
Length: 4803, dtype: object

In [313]:
vectorizer=TfidfVectorizer()
feacture_vector =vectorizer.fit_transform(combined)
feacture_vector

<4803x24882 sparse matrix of type '<class 'numpy.float64'>'
	with 116109 stored elements in Compressed Sparse Row format>

In [314]:
print(feacture_vector)

  (0, 3413)	0.17226884394189929
  (0, 18834)	0.27846462337963046
  (0, 14855)	0.15923029084316576
  (0, 12782)	0.22744360632519092
  (0, 21046)	0.1639944080455266
  (0, 23885)	0.19969024906047514
  (0, 20220)	0.20547005031078105
  (0, 19233)	0.21912635996951987
  (0, 24840)	0.201039437367968
  (0, 24480)	0.23875060876187826
  (0, 19268)	0.15850149690066337
  (0, 16640)	0.2656109171037048
  (0, 16240)	0.08837025445995489
  (0, 24458)	0.1265638623285646
  (0, 21992)	0.07607342067663711
  (0, 20542)	0.27846462337963046
  (0, 4513)	0.24941716233960082
  (0, 23733)	0.12428039342777443
  (0, 20682)	0.33719368661516197
  (0, 8601)	0.1662121750776325
  (0, 4292)	0.22166380507488503
  (0, 5228)	0.210622085695007
  (0, 7869)	0.10122396073588048
  (0, 19583)	0.1013424698095681
  (0, 7641)	0.10850554828440298
  :	:
  (4801, 19921)	0.3159725528657804
  (4801, 24660)	0.3159725528657804
  (4801, 24813)	0.3013875102973598
  (4801, 7018)	0.26186917277338256
  (4801, 579)	0.19303302447994897
  (4801, 10

In [315]:
similarity = cosine_similarity(feacture_vector)
similarity

array([[1.        , 0.07178333, 0.01418836, ..., 0.        , 0.        ,
        0.        ],
       [0.07178333, 1.        , 0.03292101, ..., 0.01222747, 0.        ,
        0.        ],
       [0.01418836, 0.03292101, 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.01222747, 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [316]:
similarity.shape

(4803, 4803)

In [317]:
movie_name = input('enter the movie:')


In [318]:
list_of_all_titles =df['title'].tolist()
list_of_all_titles

['Avatar',
 "Pirates of the Caribbean: At World's End",
 'Spectre',
 'The Dark Knight Rises',
 'John Carter',
 'Spider-Man 3',
 'Tangled',
 'Avengers: Age of Ultron',
 'Harry Potter and the Half-Blood Prince',
 'Batman v Superman: Dawn of Justice',
 'Superman Returns',
 'Quantum of Solace',
 "Pirates of the Caribbean: Dead Man's Chest",
 'The Lone Ranger',
 'Man of Steel',
 'The Chronicles of Narnia: Prince Caspian',
 'The Avengers',
 'Pirates of the Caribbean: On Stranger Tides',
 'Men in Black 3',
 'The Hobbit: The Battle of the Five Armies',
 'The Amazing Spider-Man',
 'Robin Hood',
 'The Hobbit: The Desolation of Smaug',
 'The Golden Compass',
 'King Kong',
 'Titanic',
 'Captain America: Civil War',
 'Battleship',
 'Jurassic World',
 'Skyfall',
 'Spider-Man 2',
 'Iron Man 3',
 'Alice in Wonderland',
 'X-Men: The Last Stand',
 'Monsters University',
 'Transformers: Revenge of the Fallen',
 'Transformers: Age of Extinction',
 'Oz: The Great and Powerful',
 'The Amazing Spider-Man 2',

In [319]:
find_close_match = difflib.get_close_matches(movie_name,list_of_all_titles)
find_close_match


['Gladiator']

In [320]:
close_match =find_close_match[0]
close_match

'Gladiator'

In [321]:
index_of_movie =df[df.title==close_match]['index'].values[0]
index_of_movie

274

In [322]:
similarity_score= list(enumerate(similarity[index_of_movie]))
similarity_score

[(0, 0.014421201998865596),
 (1, 0.01975216941557175),
 (2, 0.01496128125963015),
 (3, 0.010401557701973927),
 (4, 0.014168692623069693),
 (5, 0.014826107571956786),
 (6, 0.008502218549907962),
 (7, 0.014580373768328023),
 (8, 0.0086232096120493),
 (9, 0.014543297812676981),
 (10, 0.016471469795088692),
 (11, 0.013402642974421529),
 (12, 0.014820098078202695),
 (13, 0.013372785320744148),
 (14, 0.031876979636478425),
 (15, 0.007995594916175477),
 (16, 0.013931869993544059),
 (17, 0.014150063193654782),
 (18, 0.02207041260260819),
 (19, 0.030774104895859508),
 (20, 0.014059014538112157),
 (21, 0.13253899920194898),
 (22, 0.027281365493163255),
 (23, 0.021948420221642343),
 (24, 0.016275622361369337),
 (25, 0.0032554458607851355),
 (26, 0.01553498891260071),
 (27, 0.013046735871743806),
 (28, 0.014113142695725412),
 (29, 0.015225421930414916),
 (30, 0.042674307413018274),
 (31, 0.01440996673244824),
 (32, 0.007673753764398849),
 (33, 0.014662249917419518),
 (34, 0.0),
 (35, 0.01540789088

In [323]:
len(similarity_score)

4803

In [324]:
sort_movie=sorted(similarity_score,key=lambda x:x[1],reverse=True)
sort_movie

[(274, 1.0000000000000002),
 (491, 0.18804494107027916),
 (660, 0.16012397834596093),
 (541, 0.14215897645499537),
 (379, 0.13303279163928602),
 (21, 0.13253899920194898),
 (627, 0.1277989777497728),
 (2992, 0.12277376508188094),
 (1727, 0.1211006790239368),
 (2642, 0.12051433744414794),
 (2645, 0.11602217649850968),
 (3819, 0.10769831579719771),
 (331, 0.1060390323285118),
 (844, 0.10372917909023394),
 (2493, 0.1030078276034619),
 (373, 0.10211053367581305),
 (2255, 0.10171024187001149),
 (1220, 0.09870069065420482),
 (973, 0.0981254660357609),
 (1684, 0.09758150475060717),
 (622, 0.09730182575915079),
 (2166, 0.09532286077652682),
 (92, 0.09505635533887526),
 (562, 0.09468296406725842),
 (4015, 0.094576125157806),
 (3508, 0.09369773668462834),
 (564, 0.09105456183626699),
 (1019, 0.0899810112208874),
 (2225, 0.089556638642913),
 (1728, 0.08844049740465294),
 (834, 0.08752931789020744),
 (2721, 0.08615497689466536),
 (281, 0.08593310410358934),
 (2830, 0.08575611968305842),
 (189, 0.0

In [325]:
print('the movie suggested :')
i=1
for movie in sort_movie:
    index =movie[0]
    index_of_movie =df[df.index==index]['title'].values[0]
    if (i<20):
        print(i, '',index_of_movie)
        i+=1

the movie suggested :
1  Gladiator
2  Pompeii
3  Proof of Life
4  Soldier
5  Conan the Barbarian
6  Robin Hood
7  The Last Legion
8  Oliver!
9  3 Days to Kill
10  Buffalo Soldiers
11  Centurion
12  Defendor
13  Seventh Son
14  The Hunted
15  The Immigrant
16  Mission to Mars
17  The Yards
18  8MM
19  Basic
